# Vision Transformer (ViT)

[Attention Is All You Need](https://arxiv.org/abs/1706.03762)の論文が発表されてからTransformerの勢いは自然言語処理に留まらず様々な分野で高い性能を叩き出している。画像処理分野でもTransformerの波は押し寄せており、[Vision Tranformer](https://arxiv.org/abs/2010.11929)という2021年にICLRで発表された論文ではImagenetのクラス分類においてトップクラスの性能を出せるTransformerベースのモデルを提案している。

ここではそのVision Transformerのモデルをpytorchを使ってスクラッチ実装する。

#### Reference
- [Vision Transformer入門(本)](https://gihyo.jp/book/2022/978-4-297-13058-9)
    - 分かりやすくViTやViT周りの様々な論文なども紹介されている
- [huffingfaceのgihub](https://github.com/huggingface/transformers/tree/main/src/transformers/models/vit)
    - コードの参考など
    - 比較的読みやすい、、はず
- [【深層学習】Transformer - Multi-Head Attentionを理解してやろうじゃないの【ディープラーニングの世界vol.28】](https://www.youtube.com/watch?v=50XvMaWhiTY)
    - ViTではなく本家Transformerの解説動画
    - Transformerの内部の処理（特にMulti-Head Attention）が分かりやすく解説されている

### input layer
画像をパッチ分割してトークン（ベクトル）に埋め込む。  
実装としては画像をパッチ分割 -> 平坦化 -> 全結合層の一連の流れは畳み込み層を使って簡単に実装できる。

In [1]:
# モデルに必要なライブラリ
import collections
import numpy as np
import torch
import torch.nn as nn

In [2]:
# input layer
# 画像を16x16 pixelのパッチに分けそれぞれをtransformerで扱うためのトークンへと変換する

import torch.nn as nn
import collections

class ImagePatchEmbedding(nn.Module):
    """
    input: image:torch.Tensor[n,c,h,w]
    output: embedding_vector:torch.Tensor[n,p,dim]
    
    nはバッチサイズ、chwは画像のカラーチャネル、高さ、幅であり、pはトークン数（＝パッチの個数）
    dimは埋め込みベクトルの長さ（ハイパーパラメータ）
    """
    def __init__(self,image_size=224,patch_size=16,in_channel=3,embedding_dim=768):
        """
        args:
            image_size:Union[int, tuple] 画像の高さと幅
            patch_size:int 1画像パッチのピクセル幅
            in_channel:int 入力画像のチャネル数
            embeedding_dim:int トークン（埋め込みベクトル）の長さ
        """
        super().__init__()
        image_size = self._pair(image_size) # if int -> tuple
        patch_size = self._pair(patch_size)
        self.image_size = image_size
        self.patch_size = patch_size
        self.grid_size = (image_size[0]//patch_size[0], image_size[1]//patch_size[1])
        self.num_patches = self.grid_size[0]*self.grid_size[1]

        self.proj = nn.Conv2d(in_channel,embedding_dim,patch_size,patch_size)
        #self.normalize = nn.LayerNorm(embedding_dim)
    
    def forward(self,x):
        """
        x:torch.Tensor[b,c,h,w]
        -> torch.Tensor[b,p,dim]
        """
        n,c,h,w = x.shape
        assert h == self.image_size[0] and w == self.image_size[1], f'Input image size ({h}*{w}) doesn\'t match model ({self.image_size[0]}*{self.image_size[1]}).'

        x = self.proj(x)
        x = x.flatten(2).transpose(1,2) # (N,C,H,W) -> (B,P,C)
        #x = self.normalize(x)
        return x

    def _pair(self,x):
        """
        util function
        return a tuple if x is int 
        """
        return x if isinstance(x, collections.abc.Iterable) else (x, x)
        #return x if isinstance(x, tuple) else (x, x)

### multihead attention
multiheadに分割する関係上shapeが分かりづらくなるのでいちいち確認していくのが良い。  
実装上分割してるためmultihead attentionの解説でもちょんぎるような解説がなされているが直感的には分割しているのではなく、小さい複数のベクトルに埋め込んで処理を行っているという方が分かりやすい気がする。

In [3]:
import numpy as np

class MultiHeadSelfAttention(nn.Module):
    """
    input: embedding_vector:torch.Tensor[n,p,dim]
    output: embedding_vector:torch.Tensor[n,p,dim]
    """
    def __init__(self,dim,num_heads=8,qkv_bias=True,dropout=0.):
        """
        args:
            dim: int トークン（埋め込みベクトル）の長さ
            num_heads: int マルチヘッドの数
            qkv_bias: bool query,key,valueに埋め込む際の全結合層のバイアス
            dropout: float dropoutの確率
        """
        super().__init__()
        
        self.num_heads = num_heads
        assert dim % num_heads == 0, f"The hidden size {dim} is not a multiple of the number of head attention"
        self.hidden_dim = dim
        self.head_dim = dim // num_heads
        
        self.query = nn.Linear(dim,dim,bias=qkv_bias)
        self.key = nn.Linear(dim,dim,bias=qkv_bias)
        self.value = nn.Linear(dim,dim,bias=qkv_bias)
        
        self.dropout = nn.Dropout(p=dropout)
        self.projection = nn.Sequential(
            nn.Linear(dim,dim),
            nn.Dropout(p=dropout),
        )
    
    def forward(self,x):
        batch_size,num_patches,_ = x.size()

        # query,key,value (B,P,C) -> (B,P,C)
        print("x",x.shape)
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        # マルチヘッドに分割
        # (B,P,C) -> (B,Nh,P,Dh)
        multihead_qkv_shape = torch.Size([batch_size, num_patches, self.num_heads, self.head_dim])
        qs = q.view(multihead_qkv_shape)
        qs = qs.permute(0, 2, 1, 3)
        ks = k.view(multihead_qkv_shape)
        ks = ks.permute(0, 2, 1, 3)
        ks_T = ks.transpose(2,3)
        vs = v.view(multihead_qkv_shape)
        vs = vs.permute(0, 2, 1, 3)
        
        # (B,Nh,P,Dh) @ (B,Nh,Dh,P) -> (B,Nh,P,P)
        scaled_dot_product = qs@ks_T / np.sqrt(self.head_dim) 
        self_attention = nn.functional.softmax(scaled_dot_product,dim=-1)
        self_attention = self.dropout(self_attention)
        
        # (B,Nh,P,P) @ (B,Nh,P,Dh) -> (B,Nh,P,Dh)
        context_layer = self_attention@vs
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous().reshape(batch_size,num_patches,self.hidden_dim)
        # (B,P,Nh,Dh) -> (B,P,C)
        
        out = self.projection(context_layer)
        
        return out

class ViTFeedForward(nn.Module):
    """
    input: embedding_vector:torch.Tensor[n,p,dim]
    output: embedding_vector:torch.Tensor[n,p,dim]
    """
    def __init__(self,dim,hidden_dim=768*4,activation=nn.GELU(),dropout=0.):
        """
        args:
            dim: int トークン（埋め込みベクトル）の長さ
            hidden_dim: FeedForwardネットワークでの中間層のベクトルの長さ
                        慣例的にdim*4が使われている(少なくともViTでは)
                        本家では違うかも
            activation: torch.nn.modules.activation 活性化関数
            dropout: float dropoutの確率    
        """
        super().__init__()
        self.linear1 = nn.Linear(dim,hidden_dim)
        self.linear2 = nn.Linear(hidden_dim,dim)
        self.activation = activation
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self,x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.dropout(x)
        
        return x
        

class ViTBlock(nn.Module):
    def __init__(
        self,
        dim=768,
        hidden_dim=768*4,
        num_heads=12,
        activation=nn.GELU(),
        qkv_bias=True,
        dropout=0.,
    ):
        super().__init__()
        self.mhsa = MultiHeadSelfAttention(dim,num_heads,qkv_bias,dropout)
        self.ff   = ViTFeedForward(dim,hidden_dim,activation,dropout)
        self.ln = nn.LayerNorm(dim,eps=1e-10)
    
    def forward(self,x):
        """
        input: torch.Tensor[n,p,dim]
        output; torch.Tensor[n,p,dim]
        """
        z = self.ln(x)
        z = self.mhsa(z)
        x = x + z
        z = self.ln(x)
        z = self.ff(x)
        out = x + z  
        
        return out

In [4]:
class ViTEncoder(nn.Module):
    """
    todo attentionも取り出したければ取り出せるように
    """
    def __init__(
        self,
        dim=768,
        hidden_dim=768*4,
        num_heads=12,
        activation=nn.GELU(),
        qkv_bias=True,
        dropout=0.,
        num_blocks=8,
    ):
        """
        args:
            num_blocks: int ViTBlockの総数
            他は割愛
        """
        super().__init__()
        self.layer = nn.ModuleList([ViTBlock(
            dim,hidden_dim,num_heads,activation,qkv_bias,dropout
        ) for _ in range(num_blocks)])
    
    def forward(self,x):
        for i, layer_module in enumerate(self.layer):
            x = layer_module(x)
        return x

In [5]:
class VisionTransformer(nn.Module):
    def __init__(
        self,
        image_size=224,
        patch_size=16,
        in_channel=3,
        dim=768,
        hidden_dim=768*4,
        num_heads=12,
        activation=nn.GELU(),
        num_blocks=8,
        qkv_bias=True,
        dropout=0.,
        num_classes=1000,
    ):
        """
        args:
            image_size: int 入力画像の解像度
            patch_size: int パッチ分割の際の1パッチのピクセル数
            in_channel: int 入力画像のチャネル数
            dim: int トークン（埋め込みベクトル）の長さ
            hidden_dim: int FeedForward層でのベクトルの長さ
            num_heads: int マルチヘッドの数
            activation: torch.nn.modules.activation 活性化関数
            num_blocks: int ViTBlockの総数
            qkv_bias: bool query,key,valueに埋め込む際の全結合層のバイアス
            dropout: float dropoutの確率
            num_classes: int 出力次元数
        """
        super().__init__()
        
        # input layer
        self.patch_embedding = ImagePatchEmbedding(image_size,patch_size,in_channel,dim)
        num_patches = self.patch_embedding.num_patches
        self.cls_token = nn.Parameter(torch.randn(size=(1,1,dim))) # クラストークン（学習可能なパラメータ）
        self.positional_embedding = nn.Parameter(torch.randn(size=(1,num_patches+1,dim))) #　位置埋め込み（学習可能なパラメータ）
        
        # vit encoder 
        self.encoder = ViTEncoder(dim,hidden_dim,num_heads,activation,qkv_bias,dropout,num_blocks)
        
        # mlp head
        self.ln = nn.LayerNorm(dim,eps=1e-10)
        self.head = nn.Linear(dim,num_classes)
    
    def forward(self,x):
        x = self.patch_embedding(x)
        cls_token = self.cls_token.expand(x.shape[0],-1,-1)
        x = torch.cat((cls_token,x),dim=1) # (B,num_patches+1,embedding_dim)
        x = x + self.positional_embedding
        
        x = self.encoder(x)
        
        x = torch.index_select(x,1,torch.tensor(0,device=x.device))
        x = x.squeeze(1)
        x = self.ln(x)
        out = self.head(x)
        
        return out

入力としてimagenetの画像、出力として1000クラス分類を考える。  
出力が1000次元になっているのが確認できる。

In [6]:
ViT = VisionTransformer()
image = torch.randn(size=(1,3,224,224))
output = ViT(image)
output.shape # [batch_size, num_classes]

x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])
x torch.Size([1, 197, 768])


torch.Size([1, 1000])